# ENGR-E 516 Assignment 2
## Srinivas Kini
## skini@iu.edu

In [1]:
# Import spark libraries
import math
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd 
import numpy as np 

In [2]:
# Create a spark session 
spark = SparkSession.builder.appName('skini-a2').getOrCreate()
spark.sparkContext.setLogLevel('ERROR')

23/04/14 14:37:37 WARN Utils: Your hostname, Srinivass-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.229 instead (on interface en0)
23/04/14 14:37:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/14 14:37:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Part 1

In [3]:
# Read the parking violations file
ny_df = spark.read.options(header=True, inferschema=True).csv("ny_parking_violations.csv")

### Rename columns with _ for ease of use in Spark SQL

In [4]:
# Renaming columns for ease of use in Spark SQL
for column in ny_df.columns:
    new_col_name = '_'.join(column.lower().split(' '))
    ny_df = ny_df.withColumnRenamed(column, new_col_name)
ny_df.columns[:5]


['summons_number',
 'plate_id',
 'registration_state',
 'plate_type',
 'issue_date']

In [5]:
# Create a temp. VIEW for Spark SQl
table_name = "parking_violations"
ny_df.createOrReplaceTempView(table_name)

## _Q1: When are tickets most likely to be issued?_

In [6]:
print("Based on Date\n")
q1_query = f'''
SELECT
  SUBSTR(issue_date, 1, 10) AS issued_on,
  COUNT(*) AS tickets
FROM {table_name}
GROUP BY issued_on
ORDER BY tickets DESC
LIMIT 5
'''.strip()

spark.sql(q1_query).show()

print("\nBased on Year\n")
q1_query = f'''
SELECT
  YEAR(TO_DATE(CAST(UNIX_TIMESTAMP(issue_date,'MM/dd/yyyy') AS timestamp))) AS year,
  COUNT(*) AS tickets
FROM {table_name}
GROUP BY year
ORDER BY tickets DESC
LIMIT 5
'''.strip()

spark.sql(q1_query).show()



Based on Date



+----------+-------+
| issued_on|tickets|
+----------+-------+
|08/04/2022|  66726|
|08/05/2022|  65393|
|08/02/2022|  64876|
|06/30/2022|  64846|
|07/19/2022|  64815|
+----------+-------+


Based on Year



+----+-------+
|year|tickets|
+----+-------+
|2022|9154317|
|2023|2380085|
|2021|    477|
|2024|    117|
|2020|     90|
+----+-------+



## _Q2: What are the most common years and types of cars to be ticketed?_


In [17]:
q2_query = f'''
SELECT
  vehicle_year AS year,
  vehicle_make AS make,
  COUNT(*) AS num_tickets
FROM {table_name}
WHERE vehicle_year IS NOT NULL AND vehicle_year <> 0 AND vehicle_make IS NOT NULL
GROUP BY vehicle_year, vehicle_make
ORDER BY num_tickets DESC
LIMIT 5
'''.strip()

spark.sql(q2_query).show()


+----+-----+-----------+
|year| make|num_tickets|
+----+-----+-----------+
|2021|TOYOT|     117999|
|2019|HONDA|     113890|
|2021|HONDA|     107202|
|2020|HONDA|     104349|
|2022|TOYOT|      95782|
+----+-----+-----------+



## _Q3: Where are tickets most commonly issued?_

In [7]:
q3_query = '''
SELECT
  {} AS {}, COUNT(*) AS tickets
FROM parking_violations
WHERE {} IS NOT NULL
GROUP BY {}
ORDER BY tickets DESC
LIMIT 5
'''.strip()

print("By violation_county\n")
spark.sql(q3_query.format(*(["violation_county"] * 4))).show()

print("By violation_precinct\n")
spark.sql(q3_query.format(*(["violation_precinct"] * 4))).show()

print("By violation_location\n")
spark.sql(q3_query.format(*(["violation_location"] * 4))).show()

By violation_county



+----------------+-------+
|violation_county|tickets|
+----------------+-------+
|              NY|2450153|
|              QN|1858441|
|              BK|1732079|
|              BX|1497854|
|               K|1365103|
+----------------+-------+

By violation_precinct



+------------------+-------+
|violation_precinct|tickets|
+------------------+-------+
|                 0|5349526|
|                19| 282466|
|                13| 254057|
|                 6| 224686|
|               114| 221523|
+------------------+-------+

By violation_location



+------------------+-------+
|violation_location|tickets|
+------------------+-------+
|                19| 282466|
|                13| 254057|
|                 6| 224686|
|               114| 221523|
|                14| 190012|
+------------------+-------+



## _Q4: Which color of the vehicle is most likely to get a ticket?_

In [41]:
q4_query = f'''
SELECT vehicle_color AS color,
  COUNT(*) AS num_tickets
FROM parking_violations
WHERE vehicle_color IS NOT NULL
GROUP BY vehicle_color
ORDER BY num_tickets DESC
LIMIT 5
'''.strip()
spark.sql(q4_query).show()

+-----+-----------+
|color|num_tickets|
+-----+-----------+
|   GY|    2275457|
|   WH|    2055818|
|   BK|    1992788|
|   BL|     760235|
|WHITE|     671757|
+-----+-----------+



## _Based on a K-Means algorithm, please try to answer the following question: • Given a Black vehicle parking illegally at 34510, 10030, 34050 (street codes). What is the probability that it will get an ticket? (very rough prediction)._

### The first step is to filter out the data, for this data let us consider a cluster size of 4

In [8]:
ny_km = ny_df.select('vehicle_color', 'street_code1', 'street_code2', 'street_code3')

### Perform K-Means (k=4) and then show the cluster centers

In [9]:
assembler = VectorAssembler(inputCols=['street_code1', 'street_code2', 'street_code3'], outputCol="features")
ny_km = assembler.transform(ny_km)

In [10]:
kmeans = KMeans(featuresCol='features', k=4, seed=10)
fit = kmeans.fit(ny_km)
transformed = fit.transform(ny_km)
cluster_centers = fit.clusterCenters()
cluster_centers

[array([2969.14010497, 1072.02717556,  661.48646836]),
 array([50044.5112846 , 58800.13552356, 58709.93333802]),
 array([61858.56842042, 19162.26310865, 18767.0727426 ]),
 array([21476.39742951, 25580.76549152, 25929.22811102])]

### Filer out the data for the necessary street codes 

In [11]:
tran_fil = transformed.where(transformed['street_code1'].isin(34510, 10030, 34050))
tran_fil.show(5)

+-------------+------------+------------+------------+--------------------+----------+
|vehicle_color|street_code1|street_code2|street_code3|            features|prediction|
+-------------+------------+------------+------------+--------------------+----------+
|        WHITE|       10030|        6880|        6980|[10030.0,6880.0,6...|         0|
|           WH|       10030|       40404|       40404|[10030.0,40404.0,...|         3|
|         BLUE|       10030|       57490|       28230|[10030.0,57490.0,...|         3|
|           WH|       10030|       57490|       28230|[10030.0,57490.0,...|         3|
|        WHITE|       10030|       42630|        6080|[10030.0,42630.0,...|         3|
+-------------+------------+------------+------------+--------------------+----------+
only showing top 5 rows



### Next, clean the data so all codes for the color 'Black' are taken into consideration. Along with this, we will use the original dataframe to get the count of black cars and all cars. This will be used to calculate the probability grouped by the street code.

In [12]:
prob = tran_fil.groupBy('street_code1').agg(count(when(ny_df["vehicle_color"].isin('Black', 'BK.', 'BLK.', 'BLAC', 'BK', 'BLK', 'BCK', 'BC', 'BLACK'), True)).
                                              alias('n_blk'),count(ny_df["vehicle_color"]).alias('n_all'))\
                                                  .withColumn('prob', col("n_blk")/col("n_all") * 100 )
prob.show()


+------------+-----+-----+------------------+
|street_code1|n_blk|n_all|              prob|
+------------+-----+-----+------------------+
|       34050| 1556| 8515| 18.27363476218438|
|       34510| 2182|12279|17.770176724488966|
|       10030|  228| 1141| 19.98247151621385|
+------------+-----+-----+------------------+



### We can see here that the probability of a black car getting a ticket at 34050 is 18.27 %, 34510 is 17.77% and 10030 is 19.98% ###

# Part 2

In [13]:
nba = spark.read.options(header=True, inferschema=True).csv("shot_logs.csv")
nba.printSchema()

root
 |-- GAME_ID: integer (nullable = true)
 |-- MATCHUP: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- W: string (nullable = true)
 |-- FINAL_MARGIN: integer (nullable = true)
 |-- SHOT_NUMBER: integer (nullable = true)
 |-- PERIOD: integer (nullable = true)
 |-- GAME_CLOCK: timestamp (nullable = true)
 |-- SHOT_CLOCK: double (nullable = true)
 |-- DRIBBLES: integer (nullable = true)
 |-- TOUCH_TIME: double (nullable = true)
 |-- SHOT_DIST: double (nullable = true)
 |-- PTS_TYPE: integer (nullable = true)
 |-- SHOT_RESULT: string (nullable = true)
 |-- CLOSEST_DEFENDER: string (nullable = true)
 |-- CLOSEST_DEFENDER_PLAYER_ID: integer (nullable = true)
 |-- CLOSE_DEF_DIST: double (nullable = true)
 |-- FGM: integer (nullable = true)
 |-- PTS: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_id: integer (nullable = true)



## _Q1: For each pair of the players (A, B), we define the fear sore of A when facing B is the hit_
## _rate, such that B is closet defender when A is shooting. Based on the fear sore, for each_
## _player, please find out who is his ”most unwanted defender”_

### First, we will group the data by the player and the closest defender and aggregate the shots that they have missed and hit for that defender

In [15]:
# Calculate hits and misses based on SHOT_RESULT
hit_rates = nba.groupBy("player_name","closest_defender")\
    .agg(count(when(nba["SHOT_RESULT"]=="made", True))\
    .alias('hits'),count("SHOT_RESULT").alias('misses'))
hit_rates.show()

+----------------+------------------+----+------+
|     player_name|  closest_defender|hits|misses|
+----------------+------------------+----+------+
|   brian roberts|        Gasol, Pau|   0|     1|
|    al jefferson| Hardaway Jr., Tim|   0|     1|
|     cody zeller|     Price, Ronnie|   0|     1|
|       gary neal|     Beal, Bradley|   3|     3|
|       gary neal|     Smart, Marcus|   0|     4|
|gerald henderson|    Bazemore, Kent|   0|     2|
|    kemba walker|     Williams, Lou|   1|     2|
|lance stephenson|    Fournier, Evan|   0|     2|
| marvin williams| Early, Cleanthony|   1|     1|
|  gordon hayward|Aldridge, LaMarcus|   2|     5|
|  gordon hayward|    Bazemore, Kent|   1|     1|
|   trevor booker|   Thompson, Jason|   3|     3|
|   trevor booker|    Frye, Channing|   1|     2|
|     enes kanter|   Chandler, Tyson|   1|     6|
|      dante exum|      Williams, Mo|   0|     3|
|      jon ingles|     Jack, Jarrett|   1|     1|
|      jon ingles|     Williams, Lou|   1|     5|


## To calculate the hit rate, we can use the following equation
### hit_rate = hits / (hits + misses)

In [16]:
hit_rates = hit_rates.withColumn('hit_rate', round(hit_rates['hits']/(hit_rates['hits'] + hit_rates['misses']), 2))
hit_rates.orderBy(hit_rates['player_name']).show()

+------------+-----------------+----+------+--------+
| player_name| closest_defender|hits|misses|hit_rate|
+------------+-----------------+----+------+--------+
|aaron brooks|   Thompson, Klay|   1|     3|    0.25|
|aaron brooks|Livingston, Shaun|   1|     2|    0.33|
|aaron brooks|     Smith, Jason|   1|     2|    0.33|
|aaron brooks|    Lee, Courtney|   1|     3|    0.25|
|aaron brooks| Carroll, DeMarre|   1|     1|     0.5|
|aaron brooks|    Nurkic, Jusuf|   0|     2|     0.0|
|aaron brooks|     Lopez, Robin|   2|     3|     0.4|
|aaron brooks|    Harris, Devin|   0|     1|     0.0|
|aaron brooks|      Green, Jeff|   0|     1|     0.0|
|aaron brooks|     LaVine, Zach|   1|     3|    0.25|
|aaron brooks|    Ariza, Trevor|   1|     1|     0.5|
|aaron brooks|   Hayes, Charles|   0|     2|     0.0|
|aaron brooks| Williams, Marvin|   1|     2|    0.33|
|aaron brooks|  Napier, Shabazz|   4|     6|     0.4|
|aaron brooks|  Lillard, Damian|   3|     6|    0.33|
|aaron brooks|      Scola, L

### To remove duplicate values for the hit_rate we will rank the data based on player_name

In [19]:
mud = hit_rates.withColumn("rank",row_number().over(Window.partitionBy("player_name")\
    .orderBy(hit_rates["hit_rate"].asc())))
mud.show()

+------------+--------------------+----+------+--------+----+
| player_name|    closest_defender|hits|misses|hit_rate|rank|
+------------+--------------------+----+------+--------+----+
|aaron brooks|       Nurkic, Jusuf|   0|     2|     0.0|   1|
|aaron brooks|       Harris, Devin|   0|     1|     0.0|   2|
|aaron brooks|         Green, Jeff|   0|     1|     0.0|   3|
|aaron brooks|      Hayes, Charles|   0|     2|     0.0|   4|
|aaron brooks|         Scola, Luis|   0|     1|     0.0|   5|
|aaron brooks|         Exum, Dante|   0|     3|     0.0|   6|
|aaron brooks|          Lawson, Ty|   0|     5|     0.0|   7|
|aaron brooks|     Crawford, Jamal|   0|     1|     0.0|   8|
|aaron brooks|      Fournier, Evan|   0|     1|     0.0|   9|
|aaron brooks|       O'Quinn, Kyle|   0|     1|     0.0|  10|
|aaron brooks|        Wear, Travis|   0|     1|     0.0|  11|
|aaron brooks|   Dos Santos, Atila|   0|     1|     0.0|  12|
|aaron brooks|        Hairston, PJ|   0|     1|     0.0|  13|
|aaron b

### Next, we will only get those ranks which are 1 to get the 'Most Unwanted Defender'

In [20]:
mud = mud.where(mud['rank'] == 1).withColumnRenamed("closest_defender", "most_unwanted_defender")\
    .select("player_name", "most_unwanted_defender", "rank")
mud.show()

+----------------+----------------------+----+
|     player_name|most_unwanted_defender|rank|
+----------------+----------------------+----+
|    aaron brooks|         Nurkic, Jusuf|   1|
|    aaron gordon|        Rivers, Austin|   1|
| al farouq aminu|        Johnson, James|   1|
|      al horford|           Diaw, Boris|   1|
|    al jefferson|     Hardaway Jr., Tim|   1|
|   alan anderson|            Leuer, Jon|   1|
|     alan crabbe|      Sefolosha, Thabo|   1|
|        alex len|       Knight, Brandon|   1|
|   alexis ajinca|          Meeks, Jodie|   1|
|      alonzo gee|          Korver, Kyle|   1|
|amare stoudemire|            Deng, Luol|   1|
|    amir johnson|         Grant, Jerami|   1|
|  andre drummond|         James, LeBron|   1|
|  andre iguodala|           Lowry, Kyle|   1|
|    andre miller|          Turner, Evan|   1|
|  andre roberson|        Ginobili, Manu|   1|
|    andrew bogut|          Ibaka, Serge|   1|
|  andrew wiggins|        Sanders, Larry|   1|
| anthony ben

## _Q2: For each player, we define the comfortable zone of shooting is a matrix of,_
## _{SHOT DIST, CLOSE DEF DIST, SHOT CLOCK}_
## _Please develop a Spark-based algorithm to classify each player’s records into 4 comfort-_
## _able zones. Considering the hit rate, which zone is the best for James Harden, Chris Paul, Stephen Curry, and Lebron James._

### To calculate comfortable zones for these players, we can make use of K-Means clustering where each cluster maps to a zone. Since we need 4 zones, there will be 4 clusters

In [21]:
zones = nba.select('player_name','shot_result','shot_dist','close_def_dist','shot_clock').dropna().dropDuplicates()
zones.show()

+--------------------+-----------+---------+--------------+----------+
|         player_name|shot_result|shot_dist|close_def_dist|shot_clock|
+--------------------+-----------+---------+--------------+----------+
|       brian roberts|     missed|     19.6|           4.6|      16.7|
|       brian roberts|     missed|     23.8|           5.5|      17.6|
|       brian roberts|       made|     20.5|           5.3|      14.4|
|       brian roberts|     missed|     24.5|           5.2|      15.3|
|        al jefferson|       made|     19.3|           7.6|      10.0|
|        al jefferson|       made|      7.7|           2.2|       3.8|
|         cody zeller|     missed|     18.1|           4.9|      12.1|
|         cody zeller|     missed|      3.1|           2.4|      24.0|
|           gary neal|       made|     17.4|           3.1|      11.0|
|           gary neal|     missed|     25.5|           5.5|      11.8|
|           gary neal|     missed|      6.5|           0.2|      14.8|
|    g

In [22]:
assembler = VectorAssembler(inputCols=['shot_dist','close_def_dist','shot_clock'], outputCol="features", handleInvalid="skip")
kmeans = KMeans(featuresCol='features', k=4, seed=10)    
t_data = assembler.transform(zones)
op_fit = kmeans.fit(t_data)
zones = op_fit.transform(t_data)

In [23]:
zones.show()

+--------------------+-----------+---------+--------------+----------+---------------+----------+
|         player_name|shot_result|shot_dist|close_def_dist|shot_clock|       features|prediction|
+--------------------+-----------+---------+--------------+----------+---------------+----------+
|       brian roberts|     missed|     19.6|           4.6|      16.7|[19.6,4.6,16.7]|         1|
|       brian roberts|     missed|     23.8|           5.5|      17.6|[23.8,5.5,17.6]|         1|
|       brian roberts|       made|     20.5|           5.3|      14.4|[20.5,5.3,14.4]|         1|
|       brian roberts|     missed|     24.5|           5.2|      15.3|[24.5,5.2,15.3]|         1|
|        al jefferson|       made|     19.3|           7.6|      10.0|[19.3,7.6,10.0]|         2|
|        al jefferson|       made|      7.7|           2.2|       3.8|  [7.7,2.2,3.8]|         0|
|         cody zeller|     missed|     18.1|           4.9|      12.1|[18.1,4.9,12.1]|         1|
|         cody zelle

### Next, we can calculate the hit_rate like we did for the previous question

In [24]:
zones = zones.groupBy("prediction")\
    .agg(count(when(zones["SHOT_RESULT"]=="made", True))\
    .alias('hits'),count("SHOT_RESULT").alias('misses'))
zones = zones.withColumn('hit_rate', round(zones['hits']/(zones['hits'] + zones['misses']), 2))

### From the previous dataframe, let us fetch the hit rates for James Harden, Chris Paul, Stephen Curry, and Lebron James for all defenders and then calculate the Average hit_rate for them.

In [25]:
goats = hit_rates.groupBy(hit_rates['player_name'])\
    .agg(round(avg(hit_rates['hit_rate']), 2))\
    .where(hit_rates['player_name']\
    .isin('james harden', 'stephen curry', 'lebron james', 'chris paul'))
    
goats.show()
zones.show()

+-------------+-----------------------+
|  player_name|round(avg(hit_rate), 2)|
+-------------+-----------------------+
| lebron james|                   0.31|
|   chris paul|                   0.28|
| james harden|                   0.28|
|stephen curry|                   0.29|
+-------------+-----------------------+

+----------+-----+------+--------+
|prediction| hits|misses|hit_rate|
+----------+-----+------+--------+
|         1|12692| 32309|    0.28|
|         3|15779| 26674|    0.37|
|         2|10589| 28652|    0.27|
|         0|16801| 34821|    0.33|
+----------+-----+------+--------+



## From the 2 data frames above we can see that LeBron James belongs to cluster (zone) 0 where was Chris Paul, James Harden and Stephen Curry belong to cluster (zone) 1